# a. 使用大模型 API 对视频进行快速摘要（音频处理）- 完整版
> [HW9: Quick Summary of Lecture Video (演講影片快速摘要)](https://colab.research.google.com/drive/1Ysr25kz6lP7gR8DNTkJMAqOuMp2bhXes?usp=sharing#scrollTo=UULEr1GpDAl6)中文镜像版 | [PDF](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/GenAI_PDF/HW9.pdf)
> 
> 这里的变量名与 HW9 基本保持一致。
>
> 指导文章：[15. 用 API 实现 AI 视频摘要：动手制作属于你的 AI 视频助手](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/15.%20用%20API%20实现%20AI%20视频摘要：动手制作属于你的%20AI%20视频助手.md)
> 

**目标：** 学习如何使用现成的 API 快速构建语音识别相关的应用。

![Pipeline](../Guide/assets/image-20240926154540268.png)

注意，这里没有视觉上的识别。

如果你遇到了 443 或者代理端口错误，请查看[《a. 使用 HFD 加快 Hugging Face 模型和数据集的下载》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/a.%20使用%20HFD%20加快%20Hugging%20Face%20模型和数据集的下载.md#可能存在的问题443-和-git-clone-failed)。

你也可以尝试运行 [b. 精简版](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Demos/13b.%20轻松开始你的第一次%20AI%20视频总结（API%20版）-%20精简版.ipynb)，适合一键执行查看效果，其中只保留了核心代码。

在线链接（精简版）：[Kaggle](https://www.kaggle.com/code/aidemos/13b-ai-api) | [Colab](https://colab.research.google.com/drive/1yh2J4_Cy45fqvydH34LLtDpw10kuWutO?usp=sharing)

这里还有一个简单的 [🎡 AI Summarizer 脚本](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/CodePlayground/summarizer.py)供你尝试，命令行执行：
```bash
python summarizer.py ./example_video.mp4
```

## 第1部分 - 准备

### 提供的作业演讲视频

(1) 为了方便处理，已将其转换为 MP3 文件。

(2) 如果你想查看原始视频，请点击以下链接：

- 李琳山教授 信号与人生 (2023)

  - 视频：[YouTube](https://www.youtube.com/watch?v=MxoQV4M0jY8) | [Bilibili](https://www.bilibili.com/video/BV14P411B7Le)


(3) 由于原始演讲视频较长，选取了从 1:43:24 到 2:00:49 的片段用于此次演示。

### 安装必要的库

这里需要一点时间。

注意，过去的 Colab 的版本在本地运行可能存在一些问题，你可以参考下方指示进行修改：
1. 下载的OpenCC==1.1.7可能会遇到报错：
>ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found

可以选择将版本降到 1.1.6 避免这个问题，解决方法来源: [OpenCC Issues#832](https://github.com/BYVoid/OpenCC/issues/832)

2. 本地运行还可能遇到：
> ImportError: To support decoding audio files, please install 'librosa' and 'soundfile'.

对应增加安装指令即可。


In [1]:
!uv add srt
!uv add --upgrade datasets[audio]
!uv add DateTime
!uv add OpenCC
!uv add opencv-contrib-python
!uv add opencv-python
!uv add opencv-python-headless
!uv add openpyxl
!uv add openai
!uv add openai-whisper
!uv add numpy
!uv add soundfile
!uv add ipywidgets
!uv add librosa
!uv add torchcodec==0.5  # 注意，该库需要匹配torch版本进行安装：https://github.com/meta-pytorch/torchcodec#installing-torchcodec
#!uv add 'httpx<0.28.0'  # 降级 httpx 以解决关键字 'proxies' 被移除的问题，最新的 openai 库不会引发该问题，故默认注释

Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 3.16s                                       
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 3ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms
Resolved 244 packages in 1ms
Audited 229 packages in 3ms


### 导入

In [2]:
# 标准库
import os
import time
import re
import pathlib
import textwrap
import datetime

# 第三方库
import numpy as np
import srt
import soundfile as sf
from opencc import OpenCC
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, Markdown

# 项目相关库（如语音识别和API调用等）
import whisper
from datasets import load_dataset
from openai import OpenAI

### 下载数据

已经上传至 data 文件夹中，不需要再下载。

In [3]:
# 加载数据集
# dataset_name = "kuanhuggingface/NTU-GenAI-2024-HW9"
# dataset = load_dataset(dataset_name)


# 加载本地 Parquet 格式的数据集
dataset = load_dataset('parquet', data_files={'test': './data/13/test-00000-of-00001.parquet'})

In [4]:
# 准备音频
input_audio = dataset["test"][0]["audio"]
input_audio_name = dataset["test"][0]["file"]
samples = input_audio.get_all_samples()
input_audio_array = samples.data.numpy().astype(np.float32)
sampling_rate = samples.sample_rate

print(f"现在我们将转录音频: 李琳山教授 信号与人生 (2023) ({input_audio_name})。")

现在我们将转录音频: 李琳山教授 信号与人生 (2023) (NTU-GenAI-2024-HW9.mp3)。


#### 直接加载 .mp3 文件（二选一）


In [5]:
import librosa

# 指定音频文件路径
mp3_file_path = './data/13/audio.mp3'

input_audio_name = os.path.basename(mp3_file_path)

# 加载音频文件，指定采样率为 16000
input_audio_array, sampling_rate = librosa.load(mp3_file_path, sr=16000)

# 打印音频数据的采样率和形状，确保加载成功
print(f"采样率: {sampling_rate}")
print(f"音频数据形状: {input_audio_array.shape}")

print(f"现在我们将转录音频: ({input_audio_name})。")

采样率: 16000
音频数据形状: (16720000,)
现在我们将转录音频: (audio.mp3)。


## 第2部分 - 自动语音识别 (ASR)

下图是处理的样例过程，对应于之后所使用的音频：

![image-20240926155512340](../Guide/assets/image-20240926155512340.png)


### 定义语音识别函数

函数 `speech_recognition()` 旨在将音频转换为字幕。

In [6]:
def speech_recognition(model_name, input_audio, output_subtitle_path, decode_options, cache_dir="./"):
    """
    将音频转换为字幕（使用 OpenAI Whisper 模型）。

    参数:
        model_name (str): 模型名称。Whisper 提供五种模型尺寸，包括 tiny, base, small, medium, large-v3
            例如，你可以使用 'tiny'、'base'、'small'、'medium'、'large-v3' 来指定模型名称
            详细信息请见：https://github.com/openai/whisper

        input_audio (Union[str, np.ndarray, torch.Tensor]): 输入音频，可以是音频文件路径，也可以是音频波形
            - 如果输入是音频路径，例如 'input.wav'，可以直接传入 'input.wav'；
            - 如果输入是音频数组，例如变量名为 'audio_array'，则可以直接传入 'audio_array'

        output_subtitle_path (str): 输出字幕文件路径
            例如，如果你想将字幕文件保存为 'output.srt'，可以使用 'output.srt' 作为输出路径

        decode_options (dict): 转录/解码选项。当前函数中会使用到以下键：
            - language (str): 指定音频的语言，例如 'zh'、'en' 等
            - initial_prompt (str 或 None): 提供给模型的初始文本 prompt
                这可以用于提供上下文（例如专有名词、术语等），以提高识别准确性
                默认值通常为 None
            - temperature (float): 采样温度。值越高，随机性越强；值越低，输出越稳定
                默认值通常为 0.0

            更多可用选项及其含义，可以参考：
            https://github.com/openai/whisper/blob/main/whisper/decoding.py
            https://github.com/openai/whisper/blob/main/whisper/transcribe.py

        cache_dir (str): 用于缓存/保存模型权重的目录路径
            例如，你可以使用 'cache' 来指定缓存目录
            默认值: "./"

    示例:
        如果你想使用 'base' 模型将 'input.wav' 转换为 'output.srt'，并将缓存文件保存在
        'cache' 目录中，可以按如下方式调用此函数：

            decode_options = {
                "language": "zh",
                "initial_prompt": None,
                "temperature": 0.0,
            }

            speech_recognition(
                model_name="base",
                input_audio="input.wav",
                output_subtitle_path="output.srt",
                decode_options=decode_options,
                cache_dir="cache",
            )
    """

    # 记录开始时间
    start_time = time.time()

    print(f"=============== 正在加载 Whisper-{model_name} ===============")

    # 加载模型
    model = whisper.load_model(name=model_name, download_root=cache_dir)

    print(f"=============== 正在转录音频 ===============")

    # 转录音频
    transcription = model.transcribe(audio=input_audio, language=decode_options["language"], verbose=False,
                                     initial_prompt=decode_options["initial_prompt"], temperature=decode_options["temperature"])

    # 记录结束时间
    end_time = time.time()

    print(f"语音识别过程耗时 {end_time - start_time} 秒。")

    subtitles = []
    # 将转录内容转换为字幕并遍历所有片段
    for i, segment in tqdm(enumerate(transcription["segments"])):

        # 将开始时间转换为字幕格式
        start_time = datetime.timedelta(seconds=segment["start"])

        # 将结束时间转换为字幕格式
        end_time = datetime.timedelta(seconds=segment["end"])

        # 获取字幕文本
        text = segment["text"]

        # 将字幕添加到字幕列表中
        subtitles.append(srt.Subtitle(index=i, start=start_time, end=end_time, content=text))

    # 将字幕列表转换为字幕内容
    srt_content = srt.compose(subtitles)

    print(f"\n=============== 正在将字幕保存到 {output_subtitle_path} ===============")

    # 将字幕内容保存到文件中
    with open(output_subtitle_path, "w", encoding="utf-8") as file:
        file.write(srt_content)

    print(f"\n=============== 字幕保存完成 ===============")

### 设置参数

注意，这里设置的参数都是 Whisper 相关的，与后续的 AI 摘要不同。

你可以通过 model_name 设置不同的模型，使用**标识**指定。通过[官方仓库](https://github.com/openai/whisper)所提供的数据，我们可以看到不同模型需要的显存大小：

| Size   | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
| ------ | ---------- | ------------------ | ------------------ | ------------- | -------------- |
| tiny   | 39 M       | `tiny.en`          | `tiny`             | ~1 GB         | ~32x           |
| base   | 74 M       | `base.en`          | `base`             | ~1 GB         | ~16x           |
| small  | 244 M      | `small.en`         | `small`            | ~2 GB         | ~6x            |
| medium | 769 M      | `medium.en`        | `medium`           | ~5 GB         | ~2x            |
| large  | 1550 M     | N/A                | `large`            | ~10 GB        | 1x             |

**解释：**

- **Size (大小)**：表示模型的尺寸，不同大小的模型训练时使用的数据量不同，因此性能和精度也不同。较大的模型通常会有更高的精度。`Medium` 是个不错的选择，`tiny` 和 `base` 效果一般，用于学习的话也可以。
- **Parameters (参数量)**：模型的参数数量，表示模型的复杂度。参数越多，模型的性能通常越好，但也会占用更多的计算资源。
- **English-only model (仅限英文模型)**：模型的**标识**名称，只用于处理英文音频转录，适用于仅需要处理英文语音的场景。
- **Multilingual model (多语言模型)**：模型的**标识**名称，用于在代码中加载相应的模型，对应于接下来的 `model_name` 参数。
- **Required VRAM (所需显存)**：指运行该模型时所需的显存大小。如果你对参数和显存的对应关系感兴趣，可以阅读之前的文章：[《07. 探究模型参数与显存的关系以及不同精度造成的影响.md》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/07.%20探究模型参数与显存的关系以及不同精度造成的影响.md)。
- **Relative speed (相对速度)**：相对速度表示模型处理语音转录任务的效率。数字越大，模型处理速度越快，与模型的参数量成反比。

直接运行代码，交互式修改所需的参数及输入文件路径，不用关心这里的代码细节，除非你对交互感兴趣。

注意：配置完记得点击**提交配置**

In [7]:
# ===== 在此代码块中，你可以修改所需的参数及输入文件路径 =====
# 模型名称部分
model_name_markdown = Markdown("**model_name**<br>请选择要使用的模型名称：")
model_name_widget = widgets.Dropdown(
    options=['tiny', 'base', 'small', 'medium', 'large-v3'],
    value='medium',
    description='model_name:',
)

# 输出文件的后缀部分
suffix_markdown = Markdown("**suffix**<br>设置输出文件名的后缀：")
suffix_widget = widgets.Text(
    value='信号与人生',
    description='suffix:',
)

# 输出字幕文件和原始文本文件路径部分
output_subtitle_path_widget = widgets.Text(
    value=f"./output-信号与人生.srt",
    description='字幕路径:',
    disabled=True  # 禁用手动修改
)

output_raw_text_path_widget = widgets.Text(
    value=f"./output-信号与人生.txt",
    description='原始文本路径:',
    disabled=True  # 禁用手动修改
)

# 模型和数据集缓存目录部分
cache_dir_markdown = Markdown("**cache_dir**<br>设置模型和数据集缓存的目录路径：")
cache_dir_widget = widgets.Text(
    value='./',
    description='cache_dir:',
)

# 演讲视频的语言部分
language_markdown = Markdown("**language**<br>设置演讲视频的语言：")
language_widget = widgets.Text(
    value='zh',
    description='language:',
)

# 用于提供初始文本prompt（可选）
initial_prompt_markdown = Markdown("**initial_prompt**<br>用于提供初始文本prompt（可选）：")
initial_prompt_widget = widgets.Text(
    value='请用中文',
    description='initial_prompt:',
)

# 采样温度部分
temperature_markdown = Markdown("**temperature**<br>值越高，随机性越强：")
temperature_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='temperature:',
    continuous_update=False
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并更新路径
def on_button_click(b):
    global model_name, suffix, output_subtitle_path, output_raw_text_path, cache_dir, language, initial_prompt, temperature
    with output_area:  # 使用 Output 小部件捕获输出
        output_area.clear_output()  # 清除之前的输出
        model_name = model_name_widget.value
        suffix = suffix_widget.value
        output_subtitle_path = f"./output-{suffix}.srt"  # 更新全局变量
        output_raw_text_path = f"./output-{suffix}.txt"  # 更新全局变量
        cache_dir = cache_dir_widget.value
        language = language_widget.value
        initial_prompt = initial_prompt_widget.value
        temperature = temperature_widget.value
        
        # 更新字幕文件路径和原始文本路径
        output_subtitle_path_widget.value = output_subtitle_path
        output_raw_text_path_widget.value = output_raw_text_path
        
        # 打印配置
        print(f"模型名称: {model_name}")
        print(f"输出文件后缀: {suffix}")
        print(f"字幕文件路径: {output_subtitle_path}")
        print(f"原始文本路径: {output_raw_text_path}")
        print(f"缓存目录: {cache_dir}")
        print(f"语言: {language}")
        print(f"初始prompt: {initial_prompt}")
        print(f"采样温度: {temperature}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(model_name_markdown, model_name_widget)
display(suffix_markdown, suffix_widget)
display(output_subtitle_path_widget, output_raw_text_path_widget)
display(cache_dir_markdown, cache_dir_widget)
display(language_markdown, language_widget)
display(initial_prompt_markdown, initial_prompt_widget)
display(temperature_markdown, temperature_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**model_name**<br>请选择要使用的模型名称：

Dropdown(description='model_name:', index=3, options=('tiny', 'base', 'small', 'medium', 'large-v3'), value='m…

**suffix**<br>设置输出文件名的后缀：

Text(value='信号与人生', description='suffix:')

Text(value='./output-信号与人生.srt', description='字幕路径:', disabled=True)

...

Text(value='请用中文', description='initial_prompt:')

**temperature**<br>值越高，随机性越强：

FloatSlider(value=0.0, continuous_update=False, description='temperature:', max=1.0)

Button(description='提交配置', style=ButtonStyle())

Output()

**构建 DecodingOptions**

In [8]:
decode_options = {
    "language": language,
    "initial_prompt": initial_prompt,
    "temperature": temperature
}

**打印完整配置**

In [9]:
message = "转录 李琳山教授 信号与人生 (2023)"
print(f"设置: (1) 模型: whisper-{model_name} (2) 语言: {language} (3) 初始prompt: {initial_prompt} (4) 温度: {temperature}")
print(message)

设置: (1) 模型: whisper-medium (2) 语言: zh (3) 初始prompt: 请用中文 (4) 温度: 0.0
转录 李琳山教授 信号与人生 (2023)


预计需要花费 4 分钟时间（包括下载和识别，识别大概1分钟）（由模型大小，网速和你的显卡决定）

**下载+识别**：

![结果](../Guide/assets/image-20240924005355363.png)

In [10]:
# 运行 ASR
speech_recognition(model_name=model_name, input_audio=input_audio_array, output_subtitle_path=output_subtitle_path, decode_options=decode_options, cache_dir=cache_dir)

=============== 正在加载 Whisper-medium ===============
=============== 正在转录音频 ===============


100%|██████████| 104500/104500 [01:04<00:00, 1618.53frames/s]


语音识别过程耗时 75.19185137748718 秒。


386it [00:00, 349073.17it/s]


=============== 正在将字幕保存到 ./output-信号与人生.srt ===============

=============== 字幕保存完成 ===============


**检查结果**

In [11]:
'''
打开 SRT 文件并读取其内容。
SRT 文件格式为：

[索引]
[开始时间] (小时:分钟:秒) --> [结束时间] (小时:分钟:秒)
[转录内容]
'''

with open(output_subtitle_path, 'r', encoding='utf-8') as file:
    content = file.read()

print(content)

1
00:00:00,000 --> 00:00:04,000
每次说学问是做出来的

2
00:00:06,000 --> 00:00:08,000
什么意思?

3
00:00:08,000 --> 00:00:12,000
要做才会获得学问

4
00:00:13,000 --> 00:00:16,000
你如果每天光是坐在那里听

5
00:00:17,000 --> 00:00:20,000
学问很可能是左耳进右耳出的

6
00:00:21,000 --> 00:00:23,000
你光是坐在那儿读

7
00:00:23,000 --> 00:00:26,000
学问可能从眼睛进入脑海之后就忘掉了

8
00:00:26,000 --> 00:00:29,000
如何能够学问在脑海里面

9
00:00:31,000 --> 00:00:33,000
真的变成你自己学问

10
00:00:33,000 --> 00:00:35,000
就是要做

11
00:00:36,000 --> 00:00:39,000
可能有很多同学有这个经验

12
00:00:39,000 --> 00:00:41,000
你如果去修某一门课

13
00:00:41,000 --> 00:00:44,000
或者做某一个实验

14
00:00:44,000 --> 00:00:47,000
在期末就是要教一个final project

15
00:00:48,000 --> 00:00:50,000
那个final project就是要你把

16
00:00:51,000 --> 00:00:53,000
学到的很多东西

17
00:00:53,000 --> 00:00:56,000
最后整合在你的final project里面

18
00:00:56,000 --> 00:00:58,000
最后做出来的时候

19
00:00:58,000 --> 00:01:00,000
就是把它们都整合了

20
00:01:00,000 --> 00:01:02,000
当你学期结束

21
00:01:02,000 --> 00:01:04,000
真的把final project做完的时候

22
00:01:04,000 --> 00:01:05,000


In [12]:
def extract_and_save_text(srt_filename, output_filename, convert_to_tradition_chinese=False):
    """
    提取SRT文件中的文本并将其保存到新的文本文件中。
    
    如果 convert_to_tradition_chinese 为 True，则将简体中文转换为繁体中文。
    
    参数:
        srt_filename: SRT文件的路径
        output_filename: 输出文本文件的名称
        convert_to_tradition_chinese: 布尔值，指示是否将简体中文转换为繁体中文。默认值为 False
    
    示例:
        如果你的SRT文件名为 'subtitle.srt'，并且希望将提取的文本保存为名为 'output.txt' 的文件，不进行简繁转换，可以按如下方式使用该函数:
            extract_and_save_text('subtitle.srt', 'output.txt')
        
        如果需要将简体中文转换为繁体中文，可以设置 convert_to_tradition_chinese=True:
            extract_and_save_text('subtitle.srt', 'output.txt', convert_to_tradition_chinese=True)
    """

    # 打开SRT文件并读取其内容
    with open(srt_filename, 'r', encoding='utf-8') as file:
        content = file.read()

    # 使用正则表达式去除时间码
    pure_text = re.sub(r'\d+\n\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\n', '', content)

    # 去除空行
    pure_text = re.sub(r'\n\n+', '\n', pure_text)

    # 如果需要转换为繁体中文
    if convert_to_tradition_chinese:
        import opencc
        converter = opencc.OpenCC('s2t')  # s2t：简体到繁体（Traditional Chinese, Taiwan standard）
        pure_text = converter.convert(pure_text)

    # 将纯文本保存到新文件
    with open(output_filename, 'w', encoding='utf-8') as output_file:
        output_file.write(pure_text)

    print(f'提取的文本已保存到 {output_filename}')

    return pure_text

## 第3部分 - 处理自动语音识别的结果



In [13]:
def chunk_text(text, max_length):
    """
    将长字符串拆分为指定长度的较小字符串。
    
    参数:
        text: 要拆分的长字符串
        max_length: 每个较小字符串的最大长度
    
    返回:
        拆分后的字符串列表
    
    示例:
        如果你想将一个名为 "long_string" 的字符串拆分为长度为 100 的较小字符串，可以按如下方式使用该函数:
            chunk_text(long_string, 100)
    """

    return textwrap.wrap(text, max_length)

这里可以自定义修改文本块参数，修改后执行下一段代码查看结果。

如果你更偏好繁体，修改 convert_to_tradition_chinese = True。

In [14]:
# 文本块的长度
chunk_length = 512

# 决定是否转为繁体
convert_to_tradition_chinese = False

In [15]:
# 从SRT文件中提取文本并保存为新文本文件
pure_text = extract_and_save_text(srt_filename=output_subtitle_path, output_filename=output_raw_text_path, convert_to_tradition_chinese=convert_to_tradition_chinese)

# 将长文本拆分为指定长度的较小块
chunks = chunk_text(text=pure_text, max_length=512)

# 你可以查看每段的字数和内容
print("查看将长文本拆分为多个短文本的结果。\n")
for index, chunk in enumerate(chunks):
    if index == 0:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 1:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    elif index == 2:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")
    else:
        print(f"\n========== 拆分后的第 {index + 1} 段（{len(chunk)} 字） ==========\n\n")
        for text in textwrap.wrap(chunk, 80):
            print(f"{text}\n")

提取的文本已保存到 ./output-信号与人生.txt
查看将长文本拆分为多个短文本的结果。


========== 拆分后的第 1 段（506 字） ==========


每次说学问是做出来的 什么意思? 要做才会获得学问 你如果每天光是坐在那里听 学问很可能是左耳进右耳出的 你光是坐在那儿读 学问可能从眼睛进入脑海之后就忘掉了

如何能够学问在脑海里面 真的变成你自己学问 就是要做 可能有很多同学有这个经验 你如果去修某一门课 或者做某一个实验 在期末就是要教一个final

project 那个final project就是要你把 学到的很多东西 最后整合在你的final project里面 最后做出来的时候 就是把它们都整合了

当你学期结束 真的把final project做完的时候 你会忽然发现 我真的学到很多东西 那就是做出来的学问 也许有 可以举另外一个例子 就是你如果学了

某一些很复杂的演算法或者什么 好像觉得那些不见得在你的脑海里 可是后来老师出了个习题 那个习题叫你写一个很大的程式 要把所有东西都包进去

当你把这个程式写完的时候你会发现 你忽然把演算法你所有东西都弄通了 那就是学问是做出来的 所以我们永远要记得 尽量多动手多做 在动手跟做的过程之中

学问才可以变成是自己的 同样的情形就是说 很多时候这样动手或者做的 表现或者成绩 没有一个成绩单上的数字


========== 拆分后的第 2 段（512 字） ==========


使得很多人觉得那不重要 很多人甚至觉得 这门课要做final project 我就不修了太累了 或者说那门课需要 怎么样怎么样太累我就不要做了 而不知道

其实那个才是让你做的机会 然后可以学到最多 也就是说虽然很可能 那么辛苦的做很多事 没有让你获得什么具体成绩 对你的overfitting可能没有帮助

可是对你的全面学习是很有帮助 是该学的 不要漏掉这些事 这是我所说的 这个课业内可以做的这些事 刚才我们讲到思考的时候 我觉得我漏掉一点 你如果修我的信号课

你可能会发现 我上课没讲到一个数学式子的时候 我通常都不推他的 我是在解释那个数学式子在说什么话 同样的没讲到一个什么事情的时候 我通常就在解释他在说什么话

也就是说我在讲的就是 我读到课本那里的时候 我心里怎么想的 也就是我在告诉同

## 第4部分 - 摘要

这里提供的方法同 ChatGPT，因为一样使用 OpenAI 库进行。

你可以通过[《00. 大模型 API 获取步骤》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/00.%20大模型%20API%20获取步骤.md)获取 API 密钥。

其他模型可以参阅[模型广场 -- 阿里云百炼](https://bailian.console.aliyun.com/?spm=5176.29619931.J__Z58Z6CX7MY__Ll8p1ZOR.1.4d1d59fcWwSqvr#/model-market)，点击对应模型的`查看详情`。

![查看详情](../Guide/assets/image-20240924091151684.png)

在界面左上角可以看到对应的英文名称。

![对应模型名](../Guide/assets/image-20240924091414350.png)

复制它。

现在你可以随意更换为你想要的模型，不过你可能要先申请使用（通过大概要几个小时，会有短信提示）。

In [16]:
def summarization(client, summarization_prompt, model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512):
    """
    使用 OpenAI Chat API 对给定文本进行摘要。
    
    参数:
        client: OpenAI Chat API客户端
        summarization_prompt: 包含需要摘要的文本的摘要prompt
        model_name: 模型名称，默认是 "qwen-turbo"
        temperature: 控制响应的随机性。较低的值使响应更具确定性，默认是 0.0
        top_p: 通过核采样控制多样性。较高的值会导致更多样化的响应，默认是 1.0
        max_tokens: 完成任务时生成的最大tokens，默认是 512
    
    返回:
        摘要文本
    
    示例:
        如果文本是 "ABC"，摘要prompt是 "DEF"，model_name 是 "qwen-turbo"，temperature 是 0.0，top_p 是 1.0，max_tokens 是 512，则可以按如下方式调用此函数:
        
            summarization(client=client, text="ABC", summarization_prompt="DEF", model_name="qwen-turbo", temperature=0.0, top_p=1.0, max_tokens=512)
    """

    # 用户prompt是 summarization_prompt 和文本的结合
    user_prompt = summarization_prompt

    while True:

        try:
            # 使用 OpenAI Chat API 对文本进行摘要
            chat_completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
                model=model_name,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens
            )

            # 返回第一个选择的内容
            return chat_completion.choices[0].message.content

        except Exception as e:
            print(f"发生错误: {e}")
            print("3秒后重试...")
            time.sleep(3)

直接运行代码，交互式修改参数和 API，不用关心这里的代码细节。

注意：配置完记得点击**提交配置**

In [17]:
# ===== 在此代码块中，你可以修改所需的参数并设置OpenAI API密钥 =====

# 你的OpenAI API密钥部分
api_key_markdown = Markdown("**openai_api_key**<br>请输入你的 OpenAI API 密钥：")
openai_api_key_widget = widgets.Password(
    value=os.getenv('OPENAI_API_KEY'),  # 默认使用环境变量
    description='openai_api_key:',
    placeholder='请输入你的OpenAI API密钥（可以不填写，默认使用环境变量）',
    style={'description_width': 'initial'}
)

# 模型名称部分
model_name_markdown = Markdown("**model_name**<br>请选择模型名称，默认是 'qwen-turbo'：")
model_name_widget = widgets.Text(
    value='qwen-turbo',
    description='model_name:',
    style={'description_width': 'initial'}
)

# 控制响应的随机性部分
temperature_markdown = Markdown("**temperature**<br>通过温度值来控制响应的随机性，较低的值使响应更具确定性：")
temperature_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='temperature (随机性):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# 控制多样性 (Top-P) 部分
top_p_markdown = Markdown("**控制多样性 (Top-P)**<br>通过 Top-P 核采样控制多样性，较高的值会导致更多样化的响应：")
top_p_widget = widgets.FloatSlider(
    value=0,
    min=0,
    max=1,
    step=0.1,
    description='Top-P (多样性):',
    continuous_update=False,
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    with output_area:
        global openai_api_key, model_name, temperature, top_p
        output_area.clear_output()  # 清除之前的输出
        openai_api_key = openai_api_key_widget.value
        model_name = model_name_widget.value
        temperature = temperature_widget.value
        top_p = top_p_widget.value
        
        # 打印配置
        print(f"OpenAI API密钥: {'*' * len(openai_api_key)}")  # 为了安全性，只显示密钥长度
        print(f"模型名称: {model_name}")
        print(f"温度 (随机性): {temperature}")
        print(f"Top-P (多样性): {top_p}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(api_key_markdown, openai_api_key_widget)
display(model_name_markdown, model_name_widget)
display(temperature_markdown, temperature_widget)
display(top_p_markdown, top_p_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**openai_api_key**<br>请输入你的 OpenAI API 密钥：

Password(description='openai_api_key:', placeholder='请输入你的OpenAI API密钥（可以不填写，默认使用环境变量）', style=TextStyle(descr…

**model_name**<br>请选择模型名称，默认是 'qwen-turbo'：

Text(value='qwen-turbo', description='model_name:', style=TextStyle(description_width='initial'))

**temperature**<br>通过温度值来控制响应的随机性，较低的值使响应更具确定性：

FloatSlider(value=0.0, continuous_update=False, description='temperature (随机性):', max=1.0, style=SliderStyle(d…

**控制多样性 (Top-P)**<br>通过 Top-P 核采样控制多样性，较高的值会导致更多样化的响应：

FloatSlider(value=0.0, continuous_update=False, description='Top-P (多样性):', max=1.0, style=SliderStyle(descrip…

Button(description='提交配置', style=ButtonStyle())

Output()

In [18]:
# 构建 OpenAI 客户端
client = OpenAI(
    api_key=openai_api_key,
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1", # 这里使用的是阿里云的大模型，如果需要使用其他平台，请参考对应的开发文档后对应修改。如果使用 GPT 的 API，删除这行就可以直接运行。
)

#### 这里演示两种摘要方式
参考：[Text Summarization of Large Documents using LangChain 🦜🔗](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/use-cases/document-summarization/summarization_large_documents_langchain.ipynb)

分别对应于 `MapReduce` 和 `Refine`，你可以通过接下来的代码来感受二者的区别。

不要修改 `<text>`，这是一个占位符，如果你不记得占位符的概念，回看[《03. 进阶指南：自定义 Prompt 提升大模型解题能力》](https://github.com/Hoper-J/AI-Guide-and-Demos-zh_CN/blob/master/Guide/03.%20进阶指南：自定义%20Prompt%20提升大模型解题能力.md#设计prompt解决数学问题)。

##### 方法一：多段摘要方法（Multi-Stage Summarization）- MapReduce

![image.png](../Guide/assets/image-20240924092040340.png)

1. 将长文本分成多个较小的部分，并分别获取每个小段落的摘要

注意：配置完记得点击**提交配置**

In [19]:
# ===== 多段摘要：段落设置 =====

# 你可以修改摘要prompt和生成的最大tokens，但不要修改 <text> 部分
instructions = Markdown("**多段摘要：段落设置**")

# 设置最大生成tokens
max_tokens_markdown = Markdown("**max_tokens**<br>设置生成的最大tokens：")
max_tokens_widget = widgets.IntText(
    value=350,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 摘要prompt模板
summarization_prompt_markdown = Markdown("**summarization_prompt_template**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_template_widget = widgets.Textarea(
    value="用 300 个字内写出这段文字的摘要，其中包括要点和所有重要细节：<text>",
    description='summarization_prompt_template:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_template_widget.value
        
        # 打印配置
        print(f"最大tokens: {max_tokens}")
        print(f"摘要prompt模板: {summarization_prompt_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_template_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**多段摘要：段落设置**

**max_tokens**<br>设置生成的最大tokens：

IntText(value=350, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt_template**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='用 300 个字内写出这段文字的摘要，其中包括要点和所有重要细节：<text>', description='summarization_prompt_template:', layout…

Button(description='提交配置', style=ButtonStyle())

Output()

In [20]:
paragraph_summarizations = []

# 首先，我们分别对每个拆分的部分进行摘要。
for index, chunk in enumerate(chunks):

    # 记录开始时间
    start = time.time()

    # 构建摘要prompt
    summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

    # 分别对每个拆分的部分进行摘要
    response = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

    # 计算执行时间，并保留两位小数
    cost_time = round(time.time() - start, 2)

    # 打印摘要及其长度
    print(f"----------------------------第 {index + 1} 段的摘要----------------------------\n")
    for text in textwrap.wrap(response, 80):
        print(f"{text}\n")
    print(f"第 {index + 1} 段摘要的长度: {len(response)}")
    print(f"生成第 {index + 1} 段摘要的时间: {cost_time} 秒。\n")

    # 记录结果
    paragraph_summarizations.append(response)

----------------------------第 1 段的摘要----------------------------

这段文字强调“学问是做出来的”，指出仅听或读难以真正掌握知识。只有通过实践，如完成课程项目或编写程序，才能将所学整合并内化为自己的理解。作者以期末项目和编程习题

为例，说明在动手过程中，知识才会真正被掌握。最终指出，真正的学问来源于实践，而非单纯的成绩数字。

第 1 段摘要的长度: 128
生成第 1 段摘要的时间: 1.46 秒。

----------------------------第 2 段的摘要----------------------------

摘要：课程中的final project虽然辛苦，但却是学习的最佳机会，虽未必带来具体成绩，却有助于全面学习。修课时应重视思考能力的培养，如阅读数学公式时要理解

其含义，而非仅记忆。老师在授课时注重解释概念，引导学生主动思考，培养独立思考的习惯和能力。真正的学习在于深入理解，而非表面掌握。

第 2 段摘要的长度: 144
生成第 2 段摘要的时间: 1.05 秒。

----------------------------第 3 段的摘要----------------------------

本文强调学习不仅限于课业，还包括课业外的各类活动。学习被定义为增长、进步与获得快乐的过程。例如打球能提升身体素质与团队协作能力；爬山和旅行能拓展视野、增长见识。

任何带来成长与愉悦的活动都值得投入，应被视为学习的一部分。关键在于持续进步与享受过程。

第 3 段摘要的长度: 123
生成第 3 段摘要的时间: 1.01 秒。

----------------------------第 4 段的摘要----------------------------

这段文字强调谈恋爱和交朋友都是学习人际互动、情感理解和沟通的重要途径，有助于个人成长。作者认为，即使没有恋爱缘分，通过交朋友或参与社团活动（如戏剧、舞蹈等）也能

获得类似收获。参与这些活动不仅能提升社交能力，还能带来个人进步，因此鼓励大家积极投入，无论是在台前还是幕后，都能有所收获。

第 4 段摘要的长度: 141
生成第 4 段摘要的时间: 1.13 秒。

----------------------------

In [21]:
# 收集之前获得的所有摘要并打印它们。

collected_summarization = ""
for index, paragraph_summarization in enumerate(paragraph_summarizations):
    collected_summarization += f"第 {index + 1} 段的摘要: {paragraph_summarization}\n"

print(collected_summarization)

第 1 段的摘要: 这段文字强调“学问是做出来的”，指出仅听或读难以真正掌握知识。只有通过实践，如完成课程项目或编写程序，才能将所学整合并内化为自己的理解。作者以期末项目和编程习题为例，说明在动手过程中，知识才会真正被掌握。最终指出，真正的学问来源于实践，而非单纯的成绩数字。
第 2 段的摘要: 摘要：课程中的final project虽然辛苦，但却是学习的最佳机会，虽未必带来具体成绩，却有助于全面学习。修课时应重视思考能力的培养，如阅读数学公式时要理解其含义，而非仅记忆。老师在授课时注重解释概念，引导学生主动思考，培养独立思考的习惯和能力。真正的学习在于深入理解，而非表面掌握。
第 3 段的摘要: 本文强调学习不仅限于课业，还包括课业外的各类活动。学习被定义为增长、进步与获得快乐的过程。例如打球能提升身体素质与团队协作能力；爬山和旅行能拓展视野、增长见识。任何带来成长与愉悦的活动都值得投入，应被视为学习的一部分。关键在于持续进步与享受过程。
第 4 段的摘要: 这段文字强调谈恋爱和交朋友都是学习人际互动、情感理解和沟通的重要途径，有助于个人成长。作者认为，即使没有恋爱缘分，通过交朋友或参与社团活动（如戏剧、舞蹈等）也能获得类似收获。参与这些活动不仅能提升社交能力，还能带来个人进步，因此鼓励大家积极投入，无论是在台前还是幕后，都能有所收获。
第 5 段的摘要: 这段文字强调，参与各类活动（如服装道具组、戏剧学会等）虽无考试成绩，但能带来显著成长与学习机会，是提升软实力的重要途径。这些经历有助于培养团队合作、沟通与领导能力，对个人发展至关重要。电机工程等专业工作往往需要团队协作，而软实力正是推动项目成功的关键。因此，不应轻视这些活动，它们虽不体现在成绩单上，却对个人综合能力提升有深远影响。
第 6 段的摘要: 这段文字强调“软实力”（soft skills）的重要性，包括沟通、协调、交朋友、说服力、团队合作和领导能力等。这些能力对成功至关重要，即使是电机工程师也需具备。虽然少数人天生具备，但多数人需通过后天努力培养。作者分享自己大学期间通过课外学习提升软技能的经历，说明这些能力可通过自我培养获得，并强调其在个人成长中的关键作用。
第 7 段的摘要: 电机工程师的职业发展黄金期在35至55岁之间，此阶段是事业上升的关键时期。早期可能因经验不足而发展较慢，后期则因年龄增长

2. 在分别获取每个小段落的摘要后，处理这些摘要以生成最终的摘要。

直接运行代码，交互式修改，不用关心这里的代码细节。

注意：配置完记得点击**提交配置**

In [22]:
# ===== 多段摘要：总摘要设置 =====

# 你可以修改摘要prompt和生成的最大tokens，但不要修改 <text> 部分
instructions = Markdown("**多段摘要：总摘要**")

# 设置生成的最大tokens
max_tokens_markdown = Markdown("**max_tokens**<br>我们设置生成的最大tokens，确保最终摘要不超过 550 个标记。")
max_tokens_widget = widgets.IntText(
    value=550,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 摘要prompt模板
summarization_prompt_markdown = Markdown("**summarization_prompt**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_widget = widgets.Textarea(
    value="在500字以内写出以下文字的简洁摘要：<text>",
    description='summarization_prompt:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_widget.value
        
        # 打印配置
        print(f"最大tokens: {max_tokens}")
        print(f"摘要prompt模板: {summarization_prompt_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示带有说明的所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**多段摘要：总摘要**

**max_tokens**<br>我们设置生成的最大tokens，确保最终摘要不超过 550 个标记。

IntText(value=550, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt**<br>你可以修改摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='在500字以内写出以下文字的简洁摘要：<text>', description='summarization_prompt:', layout=Layout(height='100px',…

Button(description='提交配置', style=ButtonStyle())

Output()

实际运行时间与 API 有关。

In [23]:
# 记录开始时间
start = time.time()

# 运行最终摘要生成
summarization_prompt = summarization_prompt_template.replace("<text>", collected_summarization)
final_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

# 计算执行时间并四舍五入到两位小数
cost_time = round(time.time() - start, 2)

# 打印摘要及其长度
print(f"----------------------------最终摘要----------------------------\n")
for text in textwrap.wrap(final_summarization, 80):
        print(f"{text}")
print(f"\n最终摘要的长度: {len(final_summarization)}")
print(f"生成最终摘要所需的时间: {cost_time} 秒。")

----------------------------最终摘要----------------------------

本文强调学习应通过实践获得，如课程项目和编程，才能真正掌握知识。学习不仅限于课堂，还包括课外活动与人际交往，有助于全面发展。软实力如沟通、团队合作对职业成功至关
重要。电机工程师的职业发展受多种因素影响，长期目标有助于实现人生价值。

最终摘要的长度: 115
生成最终摘要所需的时间: 1.01 秒。


In [24]:
# 你可以修改最终摘要的输出路径。
output_path = f"./final-summary-{suffix}-llm-multi-stage.txt"

# 如果需要将简体中文转换为繁体中文，请将此选项设置为 True；否则，设置为 False。
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # 创建一个 OpenCC 实例用于简体到繁体中文转换。
    cc = OpenCC('s2t')
    final_summarization = cc.convert(final_summarization)

# 输出最终摘要
with open(output_path, "w") as fp:
    fp.write(final_summarization)

# 显示结果
print(f"最终摘要已保存到 {output_path}")
print(f"\n===== 以下是最终摘要 ({len(final_summarization)} 字) =====\n")
for text in textwrap.wrap(final_summarization, 64):
    print(text)

最终摘要已保存到 ./final-summary-信号与人生-llm-multi-stage.txt

===== 以下是最终摘要 (115 字) =====

本文强调学习应通过实践获得，如课程项目和编程，才能真正掌握知识。学习不仅限于课堂，还包括课外活动与人际交往，有助于全面发展。软实
力如沟通、团队合作对职业成功至关重要。电机工程师的职业发展受多种因素影响，长期目标有助于实现人生价值。


##### 方法二：精炼方法（the method of Refinement) - Refine

Refinement 就是把每次的文本和之前的摘要结合起来丢给大模型，类似于迭代：

![Refinement](../Guide/assets/image-20240924092753352.png)

步骤（Pipeline）如下：
- 第1步：从一小部分数据开始，运行prompt生成初始输出。
- 第2步：对后续每个文档，将前一个输出与新文档结合输入。
- 第3步：LLM 根据新文档中的信息精炼输出。
- 第4步：此过程持续迭代，直到处理完所有文档。

对应的核心代码：
```python
        # 第1步
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, ...)
        # 第2步
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的内容: {chunk}"""
        # 第3步
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, ...)
```

直接运行代码，交互式修改，不用关心这里的代码细节。

注意：配置完记得点击**提交配置**

In [25]:
# ===== 精炼摘要的prompt设置 =====

# 你可以修改摘要prompt和最大生成tokens，但不要修改 <text> 部分
instructions = Markdown("**精炼摘要的prompt设置**")

# 设置最大生成tokens
max_tokens_markdown = Markdown("**max_tokens**<br>设置最大生成tokens")
max_tokens_widget = widgets.IntText(
    value=550,
    description='max_tokens:',
    style={'description_width': 'initial'}
)

# 初始摘要prompt
summarization_prompt_markdown = Markdown("**summarization_prompt**<br>你可以修改初始摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_widget = widgets.Textarea(
    value="请在 300 字以内，提供以下文字的简洁摘要:<text>",
    description='summarization_prompt:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 精炼摘要prompt
summarization_prompt_refinement_markdown = Markdown("**summarization_prompt_refinement**<br>你可以修改精炼摘要prompt，但不要修改 `<text>` 部分：")
summarization_prompt_refinement_widget = widgets.Textarea(
    value="请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>",
    description='summarization_prompt_refinement:',
    layout=widgets.Layout(width='500px', height='100px'),
    style={'description_width': 'initial'}
)

# 创建输出区域来显示打印内容
output_area = widgets.Output()

# 获取用户输入值并显示
def on_button_click(b):
    global max_tokens, summarization_prompt_template, summarization_prompt_refinement_template
    with output_area:
        output_area.clear_output()  # 清除之前的输出
        max_tokens = max_tokens_widget.value
        summarization_prompt_template = summarization_prompt_widget.value
        summarization_prompt_refinement_template = summarization_prompt_refinement_widget.value
        
        # 打印配置
        print(f"最大tokens: {max_tokens}")
        print(f"初始摘要prompt: {summarization_prompt_template}")
        print(f"精炼摘要prompt: {summarization_prompt_refinement_template}")

# 创建提交按钮
submit_button = widgets.Button(description="提交配置")
submit_button.on_click(on_button_click)

# 显示 Markdown 和所有小部件
display(instructions)
display(max_tokens_markdown, max_tokens_widget)
display(summarization_prompt_markdown, summarization_prompt_widget)
display(summarization_prompt_refinement_markdown, summarization_prompt_refinement_widget)

# 显示提交按钮
display(submit_button)

# 显示输出区域
display(output_area)

**精炼摘要的prompt设置**

**max_tokens**<br>设置最大生成tokens

IntText(value=550, description='max_tokens:', style=DescriptionStyle(description_width='initial'))

**summarization_prompt**<br>你可以修改初始摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='请在 300 字以内，提供以下文字的简洁摘要:<text>', description='summarization_prompt:', layout=Layout(height='100…

**summarization_prompt_refinement**<br>你可以修改精炼摘要prompt，但不要修改 `<text>` 部分：

Textarea(value='请在 500 字以内，结合原先的摘要和新的内容，提供简洁的摘要:<text>', description='summarization_prompt_refinement:', layou…

Button(description='提交配置', style=ButtonStyle())

Output()

In [26]:
paragraph_summarizations = []

# 首先，我们分别对拆分后的每个部分进行摘要。
for index, chunk in enumerate(chunks):

    if index == 0:
        # 记录开始时间
        start = time.time()

        # 构建初始摘要prompt
        summarization_prompt = summarization_prompt_template.replace("<text>", chunk)

        # 第1步: 从一小部分数据开始，运行prompt生成初始输出
        first_paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # 记录结果
        paragraph_summarizations.append(first_paragraph_summarization)

        # 计算执行时间并四舍五入到两位小数
        cost_time = round(time.time() - start, 2)

        # 打印摘要及其长度
        print(f"----------------------------第 {index + 1} 段摘要----------------------------\n")
        print(f"{paragraph_summarization}\n")
        print(f"第 {index + 1} 段摘要的长度: {len(first_paragraph_summarization)}")
        print(f"生成第 {index + 1} 段摘要所需的时间: {cost_time} 秒。\n")

    else:
        # 记录开始时间
        start = time.time()

        # 第2步：将前一个输出与新文档结合输入
        chunk_text = f"""前 {index} 段的摘要: {paragraph_summarizations[-1]}\n第 {index + 1} 段的内容: {chunk}"""

        # 构建精炼摘要prompt
        summarization_prompt = summarization_prompt_refinement_template.replace("<text>", chunk_text)

        # 第3步：LLM 被指示根据新文档中的信息精炼输出
        paragraph_summarization = summarization(client=client, summarization_prompt=summarization_prompt, model_name=model_name, temperature=temperature, top_p=top_p, max_tokens=max_tokens)

        # 记录结果
        paragraph_summarizations.append(paragraph_summarization)

        # 计算执行时间并四舍五入到两位小数
        cost_time = round(time.time() - start, 2)

        # 打印结果
        print(f"----------------------------前 {index + 1} 段摘要----------------------------\n")
        print(f"{paragraph_summarization}\n")
        print(f"前 {index + 1} 段摘要的长度: {len(paragraph_summarization)}")
        print(f"生成前 {index + 1} 段摘要所需的时间: {cost_time} 秒。\n")

    # 第4步：此过程持续迭代，直到处理完所有文档。

----------------------------第 1 段摘要----------------------------

这段文字强调了长程目标的重要性。作者希望用五年、十年甚至更长时间，投入大量努力去做自己真正热爱且有意义的事情。他认为，拥有清晰长期目标的人更容易取得成功。他提到真正影响人生的四件事，并呼吁观众点赞、订阅、转发和打赏支持其栏目。

第 1 段摘要的长度: 111
生成第 1 段摘要所需的时间: 1.19 秒。

----------------------------前 2 段摘要----------------------------

“学问是做出来的”强调实践的重要性。只听只读难以真正掌握知识，唯有通过动手实践，如完成项目或编写程序，才能将所学内化为自己的理解。在实践中，知识被整合、运用，从而真正掌握。因此，应多参与实际操作，才能让学问成为自己的能力。然而，许多学生因课程任务繁重而放弃实践机会，殊不知这些经历正是学习的关键。即使看似辛苦且无明显成果，它却能提升全面能力与思考习惯。真正的学习在于深入理解，而非表面记忆。读书时应主动思考，不断追问“这是什么”，才能培养独立思考的能力。

前 2 段摘要的长度: 227
生成前 2 段摘要所需的时间: 2.0 秒。

----------------------------前 3 段摘要----------------------------

“学问是做出来的”强调实践的重要性，唯有通过动手操作才能真正掌握知识。课业外的学习同样重要，如打球、爬山、旅行等，只要带来成长与快乐，都是有价值的学习。学习不仅是书本知识，更是生活中的不断积累与提升。

前 3 段摘要的长度: 100
生成前 3 段摘要所需的时间: 0.95 秒。

----------------------------前 4 段摘要----------------------------

“学问是做出来的”强调实践的重要性，唯有通过动手操作才能真正掌握知识。课业外的学习同样重要，如打球、爬山、旅行等，只要带来成长与快乐，都是有价值的学习。学习不仅是书本知识，更是生活中的不断积累与提升。谈恋爱、交朋友、参与社团活动等也是学习的一部分，能帮助我们理解人际关系、沟通与期待，促进个人成长。即使不直接参与表演，幕后工作也能锻炼能力，带来

In [27]:
# 你可以修改最终摘要的输出路径。

output_path = f"./final-summary-{suffix}-llm-refinement.txt"

# 如果需要将简体中文转换为繁体中文，请将此选项设置为 True；否则，设置为 False。
convert_to_tradition_chinese = False

if convert_to_tradition_chinese == True:
    # 创建一个 OpenCC 实例用于简体到繁体中文转换。
    cc = OpenCC('s2t')
    paragraph_summarizations[-1] = cc.convert(paragraph_summarizations[-1])

# 输出最终摘要
with open(output_path, "w") as fp:
    fp.write(paragraph_summarizations[-1])

# 显示结果
print(f"最终摘要已保存到 {output_path}")
print(f"\n===== 以下是最终摘要 ({len(paragraph_summarizations[-1])} 字) =====\n")
print(paragraph_summarizations[-1])

最终摘要已保存到 ./final-summary-信号与人生-llm-refinement.txt

===== 以下是最终摘要 (219 字) =====

摘要：文章强调“学问是做出来的”，主张通过实践与生活提升软实力，如沟通、团队合作与领导力，这对电机工程师至关重要。作者结合自身经历，指出课外活动有助于培养这些能力。第7段探讨职业发展，认为35至55岁是黄金期，成功取决于实力、努力、智慧与自我管理，而非仅学术成绩。第8段进一步强调实力源于全面学习，避免过度专精；努力决定长期成就；自我技能来自课业外的积累。第9段提出长程目标的重要性，认为明确目标能引导持续成长，真正影响人生的是这四件事。


## 总结

通过这个代码文件，你将了解如何：

1. 使用 Whisper 模型从视频音频生成字幕。
2. 将字幕文件中的内容提取并处理，转换为文本格式。
3. 使用 OpenAI API 为文本生成摘要。

你也将：
1. 理解 AI 视频总结助手实际上是如何工作的。
2. 发现 AI 应用实际真的并不难，你完全有能力制作一个属于自己的 AI 应用。
